## Script to bundle all resources in folder as a batch, transaction, collection, or message

- python version 3.6+
- get data from folder
- bundle
- validate and save

In [237]:
from json import load, dumps, loads
from IPython import display as D
from requests import get, post, put
from IPython.display import display, Markdown, HTML
from fhirclient.models import bundle as B
import fhirclient.models.fhirdate as D
import os, uuid
from datetime import datetime

bundle_id = 'indv-report08'
bundle_type = 'transaction'

###  write to file

In [238]:
def write_file(r): # write file
    #out_path = '/Users/ehaas/Documents/FHIR/Davinci-DEQM/source/examples/'  # append forward slash
    out_path ='r3'
    with open(f'{out_path}/Bundle-{bundle_id}.json', 'w') as f:
        f.write(r)


### open json file and return as dict

In [239]:
def open_file(f_name): # get files
    with open(f'{in_path}/{f_name}') as f:
        r = f.read()
        return(loads(r))

### Get files in current path

In [240]:
in_path = 'r3'
files = [x for x in os.listdir(in_path) if x.endswith(".json") and not x.startswith("Bundle")]
print(files)


['Coverage-coverage01.json', 'Encounter-encounter01.json', 'Location-location01.json', 'MeasureReport-indv-measurereport01.json', 'Organization-organization01.json', 'Organization-organization02.json', 'Patient-patient01.json', 'Practitioner-practitioner01.json', 'Task-task01.json']


### create Bundle 'b'  change the id for unique Bundles!!!

In [241]:
b = B.Bundle()
b.id =  bundle_id
b.type = bundle_type
#b.foo = D.FHIRDate(f'{datetime.utcnow().isoformat()}Z')
b.as_json()

{'id': 'indv-report08', 'type': 'transaction', 'resourceType': 'Bundle'}

### Add resources to bundle

#### Create a mapping of ids to uuids

- get id from resource
- map to uuid (create a dict of {urn, (old_ref)})
- remove id element
- replace all old refs in bundle with new urns


In [242]:
b.entry = []
ref_map = {}
for i in files:
    r =open_file(i)
    # remove text adn id elements from Bundle
    r.pop('text', None) # remove text elements from Bundle
    old_id = r.pop('id',None) # remove text elements from Bundle
    new_urn = uuid.uuid1().urn # new urn for resource
    ref_map[new_urn] = f'{r["resourceType"]}/{old_id}'
    # print( id_map , '\n')
    e = B.BundleEntry(dict
        (
        resource = r,
        fullUrl = new_urn,
        )
      )
    if bundle_type in ['transaction', 'batch']:
        e.request = B.BundleEntryRequest(dict
                    (
                    method = 'POST',
                    url = r['resourceType']
                    )
                    )
    
    b.entry.append(e)

b_json = dumps(b.as_json(),indent =3)
# print(b_json)
for new_ref, old_ref in ref_map.items():
    b_json = b_json.replace(old_ref,new_ref)
print(b_json)    
write_file(b_json)

{
   "id": "indv-report08",
   "entry": [
      {
         "fullUrl": "urn:uuid:fa2c897e-0bf3-11ea-b0f6-4ccc6a181ed3",
         "request": {
            "method": "POST",
            "url": "Coverage"
         },
         "resource": {
            "meta": {
               "lastUpdated": "2019-02-11T14:19:25.000+00:00",
               "profile": [
                  "http://hl7.org/fhir/us/davinci-deqm/STU3/StructureDefinition/coverage-deqm"
               ],
               "versionId": "2"
            },
            "beneficiary": {
               "reference": "urn:uuid:fa2cd76e-0bf3-11ea-a780-4ccc6a181ed3"
            },
            "payor": [
               {
                  "reference": "Organization/organization04"
               }
            ],
            "policyHolder": {
               "reference": "urn:uuid:fa2cd76e-0bf3-11ea-a780-4ccc6a181ed3"
            },
            "relationship": {
               "coding": [
                  {
                     "code": "self"
    